# CHAPTER 7. REGULARIZATION FOR DEEP LEARNING

* 딥러닝 세미나 : 이론 [1]
* 김무성

# Contents
* 7.1 Parameter Norm Penalties
* 7.2 Norm Penalties as Constrained Optimization
* 7.3 Regularization and Under-Constrained Problems
* 7.4 Dataset Augmentation
* 7.5 Noise Robustness
    - 7.5.1 Injecting Noise at the Output Targets
* 7.6 Semi-Supervised Learning
* 7.7 Multi-Task Learning
* 7.8 Early Stopping
* 7.9 Parameter Tying and Parameter Sharing
* 7.10 Sparse Representations
* 7.11 Bagging and Other Ensemble Methods
* <font color="red">7.12 Dropout</font>
* 7.13 Adversarial Training
* 7.14 Tangent Distance, Tangent Prop, and ManifoldTangent Classifier

#### 참고
* [2] Training Neural Networks Part 2: parameter updates, ensembles, dropout Convolutional Neural Networks: intro - http://cs231n.stanford.edu/slides/winter1516_lecture6.pdf

# 7.12 Dropout

* Dropout(Srivastava et al., 2014)은 비싸지 않으면서도 regularizing 하는데 강력한 방법이다.
* Dropout은 매우 크고 많은 뉴럴 네트워크에 대하여 실용적으로 bagging을 할 수 있는 방법으로 생각할 수 있다.
* bagging이란??
* 우선 ensemble 부터...
* 기존에는 아주 강한 classifier를 설계하려고 했다. 그런데 ensemble은 약하고 간단한 classifier를 많이 만들어 두고, 각 classifier에게 물어보면 각 classifier는 자기의 결과를 말해준다. 가장 많은 득표를 한 값을 결과로 도출한다. 
* 그럼 bagging은?
* weak classifier가 k개 있다고 할 때, 각 classifier를 학습할 data를 k번 bootstrapping 하여 선택하고 그것을 가지고 training 하는 방법.
* bootstrapping이란?
* training set을 선택하는데 있어서, 정해진 값을 선택하는 것이 아니라, 해당 값의 확률 분포를 modeling하고, 그 모델에서 원하는 만큼의 data를 뽑아서 사용하는 것.
* ex) 1,2,3,3,4 라는 데이터가 있으면 1,2,4가 나올 확률은 0.2, 3이 나올 확률은 0.4가 되는데 이 확률 값을 바탕으로 data를 resampling 함.
* bagging은 많은 모델들을 훈련시키고, 또 각각의 모델들을 테스트 한다.
* 그런데 만약 각각의 모델 하나하나가 거대한 뉴럴 네트워크라면 상당히 많은 비용이 발생할 것이다.
* Dropout을 통해 이러한 모델들을 보다 효율적으로 훈련하고 테스트할 수가 있다.

* 드롭 아웃은 기본 네트워크에서 비 출력 단위를 제거하여 구성 할 수있는 모든 하위 네트워크로 구성된 앙상블을 학습한다. 그림 7.6.

<img src="figures/cap7.12.1.png" width=600 />
<img src="figures/cap7.12.2.png" width=600 />

* 일반적으로 input unit에 대한 dropout 확률은 0.8, 히든 유닛에 대한 확률은 0.5로 설정한다.

<img src="figures/cap7.12.3.png" width=200 />
<img src="figures/cap7.12.4.png" width=400 />
<img src="figures/cap7.12.5.png" width=600 />

#### cost function
* More formally, suppose that a mask vector $µ$ speciﬁes which units to include, and $J(θ, µ)$ deﬁnes the cost of the model deﬁned by parameters $θ$ and mask $µ$.
* Then dropout training consists in minimizing $E_µJ(θ, µ)$.

#### parameters sharing
* Dropout training is not quite the same as bagging training. In the case of bagging, the models are all independent. 
* <font color="red">In the case of dropout, the models share parameters, with each model inheriting a diﬀerent subset of parameters from the parent neural network</font>. 

#### training 
* In the case of dropout, typically most models are not explicitly trained at all—usually,the model is large enough that it would be infeasible to sample all possible sub-networks within the lifetime of the universe. 
* <font color="red">Instead, a tiny fraction of the possible sub-networks are each trained for a single step, and the parameter sharing causes the remaining sub-networks to arrive at good settings of the parameters</font>.

#### inference
* To make a prediction, a bagged ensemble must accumulate votes from all of its members. 
* <font color="red">We refer to this process as inference in this context</font>. 
* So far, our description of bagging and dropout has <font color="red">not required that the model be explicitly probabilistic</font>. 
* Now, we assume that the model’s role is to output a probability distribution.

<font color="blue">In the case of bagging</font>, each model $i$ produces a probability distribution $p^{(i)}(y|x)$. The prediction of the ensemble is given by the arithmetic mean of all of these distributions,

<img src="figures/cap7.12.6.png" width=600 />

<font color="blue">In the case of dropout</font>, each sub-model deﬁned by mask vector $µ$ deﬁnes a probability distribution $p(y|x,µ)$. The arithmetic mean over all masks is given by

<img src="figures/cap7.12.7.png" width=600 />

where $p(µ)$ is the probability distribution that was used to sample $µ$ at training time.

##### approximation
* Because this sum includes an exponential number of terms, <font color="red">it is intractable to evaluate</font> except in cases where the structure of the model permits some form of simpliﬁcation.
    - So far, deep neural nets are not known to permit any tractable simpliﬁcation.
* <font color="red">Instead, we can approximate the inference with sampling, by averaging together the output from many masks</font>. 
    - Even 10-20 masks are often suﬃcient to obtain good performance.

##### geometric mean

* However, there is an even better approach, that <font color="red">allows us to obtain a good approximation to the predictions of the entire ensemble</font>, at the <font color="blue">cost of only one forward propagation</font>. 
* To do so, we change to <font color="red">using the geometric mean rather than the arithmetic mean of the ensemble members’ predicted distributions</font>.

#### 참고 
* [3] Geometric mean - https://en.wikipedia.org/wiki/Geometric_mean

The geometric mean of multiple probability distributions is not guaranteed to be a probability distribution. 

To guarantee that the result is a probability distribution, <font color="red">we impose the requirement that none of the sub-models assigns probability 0 to any event, and we renormalize the resulting distribution</font>.

The unnormalized probability distribution deﬁned directly by the geometric mean is given by

<img src="figures/cap7.12.8.png" width=600 />

where $d$ is the number of units that may be dropped.

To make predictions we must re-normalize the ensemble:

<img src="figures/cap7.12.9.png" width=600 />

#### weight scaling inference rule

* <font color="red">A key insight (Hinton et al., 2012c) involved in dropout is that we can approximate $p_{ensemble}$ by evaluating $p(y|x)$ in one model</font>: 
    - the model with all units, but with the weights going out of unit $i$ multiplied by the probability of including unit $i$. 
    - The motivation for this modiﬁcation is to capture the right expected value of the output from that unit. 
* We call this approach the <font color="red">weight scaling inference rule</font>.
     - There is not yet any theoretical argument for the accuracy of this approximate inference rule in deep nonlinear networks, but empirically it performs very well
    - Because we usually use an inclusion probability of 1/2, the weight scaling rule usually amounts to dividing the weights by 2 at the end of training, and then using the model as usual.
     - Another way to achieve the same result is to multiply the states of the units by 2 during training.
     - Either way, the goal is to make sure that
         - the <font color="red">expected total input</font> to a unit <font color="red">at test time</font> 
             - <font color="red">is roughly the same</font> 
                 - as the expected total input to that unit <font color="red">at train time</font>, 
                     - even though half the units at train time a remissing on average.

<img src="./figures/cap7.12.15.png">

For many classes of models that do <font color="red">not have nonlinear hidden units, the weight scaling inference rule is exact</font>. 

<font color="blue">For a simple example, consider a softmax regression classiﬁer</font> with n input variables represented by the vector v:

<img src="figures/cap7.12.10.png" width=600 />

We can index into the family of sub-models by element-wise multiplication of the input with a binary vector d:

<img src="figures/cap7.12.11.png" width=600 />

The ensemble predictor is deﬁned by re-normalizing the geometric mean over all ensemble members’ predictions:

<img src="figures/cap7.12.12.png" width=600 />

<img src="figures/cap7.12.13.png" width=600 />

<img src="figures/cap7.12.14.png" width=600 />

However, the weight scaling rule is only an approximation <font color="blue">for deep models that have nonlinearities</font>. 
* <font color="red">Though the approximation has not been theoretically characterized, it often works well, empirically</font>.

#### weight scaling rule & Monte Carlo approximations
*  Goodfellowet al. (2013a) found experimentally that the weight scaling approximation can work better (in terms of classiﬁcation accuracy) than Monte Carlo approximations to theensemble predictor. 
* Gal and Ghahramani (2015) found that some models obtain better classiﬁcation accuracy using twenty samples and the Monte Carlo approximation. 
* <font color="red">It appears that the optimal choice of inference approximation is problem-dependent</font>. 

#### advantages of dropout
* Srivastava et al. (2014) showed that dropout is more eﬀective than
    - other standard computationally inexpensive regularizers, 
        - such as 
            - weight decay, 
            - ﬁlternorm constraints and 
            - sparse activity regularization. 
* Dropout may also be combined with other forms of regularization to yield a further improvement.
* One advantage of dropout is that it is <font color="red">very computationally cheap</font>.
    - Using dropout during training requires onlyvO(n) computation per example per update, to generaten random binary numbers and multiply them by the state.
* Another signiﬁcant advantage of dropout is that it <font color="red">does not signiﬁcantly limit</font> 
    - the <font color="red">type of model</font> or 
    - <font color="red">training procedure</font> that can be used.
    - Many other regularization strategies of comparable power impose more severe restrictions on the architecture of the model.

#### cost of dropout
* Though the cost per-step of applying dropout to a speciﬁc model is negligible,the <font color="red">cost of using dropout in a complete system can be signiﬁcant</font>. 
* Because dropout is a regularization technique, <font color="red">it reduces the eﬀective capacity of a model</font>. 
* To oﬀset this eﬀect, <font color="red">we must increase the size of the model</font>.
* Typically the optimal validation set error is much lower when using dropout, but this comes at the cost of a much larger model and many more iterations of the training algorithm.
* <font color="blue">For very large datasets, regularization confers little reduction in generalization error</font>.
* <font color="red">In these cases, the computational cost of using dropout and larger models may outweigh the beneﬁt of regularization</font>.
* <font color="blue">When extremely few labeled training examples</font> are available, <font color="red">dropout is less eﬀective</font>. 

#### dropout & weight decay
* Wager et al. (2013) showed that, when applied to linear regression, dropout is equivalent to $L^2$ weight decay.
    - Similar results hold for other linear models
* For deepmodels, dropout is not equivalent to weight decay.

#### stochasticity & fast dropout & dropout boosting 

##### stochasticity

* The stochasticity used while training with dropout is not necessary for the approach’s success. 
* It is just a means of approximating the sum over all sub-models.

##### fast dropout

* Wang and Manning (2013) derived analytical approximations to this marginalization. 
* Their approximation, known as <font color="red">fast dropout</font> resulted in faster convergence time due to the reduced stochasticity in the computation of the gradient. 
    - This method can also be applied at test time, as a more principled(but also more computationally expensive) approximation to the average over all sub-networks than the weight scaling approximation.
* <font color="red">Fast dropout has been used to nearly match the performance of standard dropout on small neural network problems, but has not yet yielded a signiﬁcant improvement or been applied to a large problem</font>.

##### dropout boosting

###### 참고
* [4] 앙상블 기법 4 - 부스팅 - http://ishuca.tistory.com/entry/%EC%95%99%EC%83%81%EB%B8%94-%EA%B8%B0%EB%B2%95-4-%EB%B6%80%EC%8A%A4%ED%8C%85


* Just as stochasticity is not necessary to achieve the regularizing eﬀect of dropout, it is also not suﬃcient. 
* To demonstrate this, Warde-Farley et al. (2014)designed control experiments using a method called dropout boosting 
    - that they designed to use exactly the same mask noise as traditional dropout but lack its regularizing eﬀect. 
    - Dropout boosting trains the entire ensemble to jointly maximize the log-likelihood on the training set. 
    - In the same sense that traditional dropout is analogous to bagging, this approach is analogous to boosting. 
* As intended, experiments with dropout boosting show almost no regularization eﬀect compared to training the entire network as a single model. 
- <font color="red">This demonstrates that the interpretation of dropout as bagging has value beyond the interpretation of dropout as robustness to noise</font>. 
- The regularization eﬀect of the bagged ensemble is only achieved when the stochastically sampled ensemble members are trained to perform well independently of each other.

#### Dropout & DropConnect
* Dropout has inspired other stochastic approaches to training exponentiallylarge ensembles of models that share weights. 
* <font color="red">DropConnect</font> is a special case of dropout where each product between a single scalar weight and a single hidden unit state is considered a unit that can be dropped
* <font color="red">Stochastic pooling</font> is a form of randomized pooling (see section 9.3) for building ensembles of convolutional networks with each convolutional network attending to diﬀerentspatial locations of each feature map. 

So far, dropout remains the most widely used implicit ensemble method.

<font color="red">One of the key insights of dropout is that training a network with stochastic behavior and making predictions by averaging over multiple stochastic decisions implements a form of bagging with parameter sharing</font>. 

# 참고자료
* [1] DEEP LEARNING (Yoshua Bengio)- http://www.deeplearningbook.org/
* [2] Training Neural Networks Part 2: parameter updates, ensembles, dropout Convolutional Neural Networks: intro - http://cs2Geometric mean31n.stanford.edu/slides/winter1516_lecture6.pdf
* [3] Geometric mean - https://en.wikipedia.org/wiki/Geometric_mean
* [4] 앙상블 기법 4 - 부스팅 - http://ishuca.tistory.com/entry/%EC%95%99%EC%83%81%EB%B8%94-%EA%B8%B0%EB%B2%95-4-%EB%B6%80%EC%8A%A4%ED%8C%85